# Overview

This notebook processes grib2 files with HRRR data.  If you run every code cell in order, this notebook will do the following:

1. Download one HRRR file (init 2100 UTC 7 Sep 2022, valid 0000 UTC 8 Sep 2022 -- thus a 3-hour forecast).
2. Extract needed predictor variables from the HRRR file.
3. Save predictor variables to an xarray table and write this xarray table to a NetCDF file.

# TODO

 - Determine the sentinel value that HRRR files use for missing data.  I have not found any missing data yet.
 - Stop replying on wgrib2.  There is nothing wrong with wgrib2, but currently the Python code needs to call wgrib2 via `subprocess.call`, which creates a temporary text file that Python then needs to read in.  This is sloppy.  I should use pygrib or something similar, but I have never been able to successfully install pygrib.

# Install wgrib2

This is ugly, but it works!

In [ ]:
!wget ftp://ftp.cpc.ncep.noaa.gov/wd51we/wgrib2/wgrib2.tgz

--2022-09-08 22:38:41--  ftp://ftp.cpc.ncep.noaa.gov/wd51we/wgrib2/wgrib2.tgz
           => ‘wgrib2.tgz’
Resolving ftp.cpc.ncep.noaa.gov (ftp.cpc.ncep.noaa.gov)... 140.90.101.71
Connecting to ftp.cpc.ncep.noaa.gov (ftp.cpc.ncep.noaa.gov)|140.90.101.71|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /wd51we/wgrib2 ... done.
==> SIZE wgrib2.tgz ... 30034766
==> PASV ... done.    ==> RETR wgrib2.tgz ... done.
Length: 30034766 (29M) (unauthoritative)

wgrib2.tgz          100%[===================>]  28.64M  1.43MB/s    in 26s     

2022-09-08 22:39:11 (1.11 MB/s) - ‘wgrib2.tgz’ saved [30034766]



In [ ]:
import tarfile

tar_file_handle = tarfile.open('wgrib2.tgz')
tar_file_handle.extractall('/content/wgrib2_install')
tar_file_handle.close()

In [ ]:
!cd /content/wgrib2_install/grib2; export CC=gcc; export FC=gfortran; make

Streaming output truncated to the last 5000 lines.
 /usr/bin/install -c -m 644 'jasper.h' '/content/wgrib2_install/grib2/include/jasper/jasper.h'
 /usr/bin/install -c -m 644 'jas_config.h' '/content/wgrib2_install/grib2/include/jasper/jas_config.h'
 /usr/bin/install -c -m 644 'jas_config2.h' '/content/wgrib2_install/grib2/include/jasper/jas_config2.h'
 /usr/bin/install -c -m 644 'jas_cm.h' '/content/wgrib2_install/grib2/include/jasper/jas_cm.h'
 /usr/bin/install -c -m 644 'jas_fix.h' '/content/wgrib2_install/grib2/include/jasper/jas_fix.h'
 /usr/bin/install -c -m 644 'jas_debug.h' '/content/wgrib2_install/grib2/include/jasper/jas_debug.h'
 /usr/bin/install -c -m 644 'jas_getopt.h' '/content/wgrib2_install/grib2/include/jasper/jas_getopt.h'
 /usr/bin/install -c -m 644 'jas_icc.h' '/content/wgrib2_install/grib2/include/jasper/jas_icc.h'
 /usr/bin/install -c -m 644 'jas_image.h' '/content/wgrib2_install/grib2/include/jasper/jas_image.h'
 /usr/bin/install -c -m 644 'jas_init.h' '/content/w

In [ ]:
!chmod +x /content/wgrib2_install/grib2/wgrib2/wgrib2

# Download HRRR data

In [ ]:
import tarfile
from urllib.request import urlretrieve

TAR_FILE_NAME_ONLINE = (
    'https://storage.googleapis.com/hrrr-grib2-processing-for-fire-wx/'
    'hrrr_data_for_notebook.tar'
)
TAR_FILE_NAME = '/content/hrrr_data_for_notebook.tar'
DATA_DIR_NAME = '/content/hrrr_data_for_notebook'

print('Downloading file: "{0:s}"...'.format(TAR_FILE_NAME_ONLINE))
urlretrieve(TAR_FILE_NAME_ONLINE, TAR_FILE_NAME)

tar_file_handle = tarfile.open(TAR_FILE_NAME)
tar_file_handle.extractall(DATA_DIR_NAME)
tar_file_handle.close()

# Define constants and import libraries

In [ ]:
import os
import time
import calendar
import tempfile
import warnings
import subprocess
import numpy
import xarray

WGRIB2_EXE_NAME_DEFAULT = '/content/wgrib2_install/grib2/wgrib2/wgrib2'

KILOGRAMS_TO_MICROGRAMS = 1e6

LATITUDE_KEY_ORIG = 'latitude'
LONGITUDE_KEY_ORIG = 'longitude'
VARIABLE_NAMES_3D_ORIG = ['HGT', 'UGRD', 'VGRD', 'VVEL', 'TMP', 'SPFH']
VARIABLE_NAMES_2D_ORIG = ['HPBL', 'MASSDEN', 'COLMD']

ROW_DIMENSION = 'row'
COLUMN_DIMENSION = 'column'
PRESSURE_DIMENSION = 'pressure_mb'

LATITUDE_KEY = 'latitude_deg_n'
LONGITUDE_KEY = 'longitude_deg_e'
FORECAST_TIME_KEY = 'forecast_time_unix_sec'
VALID_TIME_KEY = 'valid_time_unix_sec'

# PRESSURE_LEVELS_MB = numpy.linspace(50, 1000, num=39, dtype=int)
PRESSURE_LEVELS_MB = numpy.array([700, 850, 1000], dtype=int)

VARIABLE_NAMES_3D = [
    'geopotential_height_m_asl', 'zonal_wind_m_s01', 'meridional_wind_m_s01',
    'vertical_velocity_pa_s01', 'temperature_kelvins',
    'specific_humidity_kg_kg01'
]
VARIABLE_NAMES_2D = [
    'boundary_layer_height_m_asl', 'smoke_density_8magl_micrograms_m03',
    'smoke_density_total_column_micrograms_m03'
]
CONVERSION_RATES_2D = numpy.array([
    1, KILOGRAMS_TO_MICROGRAMS, KILOGRAMS_TO_MICROGRAMS
])

# Define helper methods

In [ ]:
def string_to_unix_sec(time_string, time_directive):
    """Converts time from string to Unix format.

    Unix format = seconds since 0000 UTC 1 Jan 1970.

    :param time_string: Time string.
    :param time_directive: Format of time string (examples: "%Y%m%d" if string
        is "yyyymmdd", "%Y-%m-%d-%H%M%S" if string is "yyyy-mm-dd-HHMMSS",
        etc.).
    :return: unix_time_sec: Time in Unix format.
    """

    time_string = str(time_string)
    time_directive = str(time_directive)

    return calendar.timegm(time.strptime(time_string, time_directive))


def unix_sec_to_string(unix_time_sec, time_directive):
    """Converts time from Unix format to string.

    Unix format = seconds since 0000 UTC 1 Jan 1970.

    :param unix_time_sec: Time in Unix format.
    :param time_directive: Format of time string (examples: "%Y%m%d" if string
        is "yyyymmdd", "%Y-%m-%d-%H%M%S" if string is "yyyy-mm-dd-HHMMSS",
        etc.).
    :return: time_string: Time string.
    """

    unix_time_sec = int(numpy.round(unix_time_sec))
    time_directive = str(time_directive)

    return time.strftime(time_directive, time.gmtime(unix_time_sec))


def read_model_coords(hdf5_file_name):
    """Reads model coordinates from HDF5 file.

    M = number of rows in grid
    N = number of columns in grid

    :param hdf5_file_name: Path to input file.
    :return: latitude_matrix_deg_n: M-by-N numpy array of latitudes (deg north).
    :return: longitude_matrix_deg_e: M-by-N numpy array of longitudes (deg
        east).
    """

    coord_table_xarray = xarray.open_dataset(hdf5_file_name)
    return (
        coord_table_xarray[LATITUDE_KEY_ORIG].values,
        coord_table_xarray[LONGITUDE_KEY_ORIG].values
    )


def read_field_from_grib2(
        grib2_file_name, field_name, num_grid_rows, num_grid_columns,
        wgrib2_exe_name=WGRIB2_EXE_NAME_DEFAULT, raise_error_if_fails=True):
    """Reads one field from grib2 file.

    One field = one variable at one vertical level at one time step

    M = number of rows in grid
    N = number of columns in grid

    :param grib2_file_name: Path to input file.
    :param field_name: Name of field to extract.  For example, 500-mb height is
        "HGT:500 mb".
    :param num_grid_rows: Number of rows in grid.
    :param num_grid_columns: Number of columns in grid.
    :param wgrib2_exe_name: Path to wgrib2 executable on local machine.
        (Sorry, I've never managed to successfully install pygrib or any other
        Python tool for processing grib files, so my hack solution is to call
        wgrib2 outside of Python.)
    :param raise_error_if_fails: Boolean flag.  If True and this method fails,
        will raise an error.  If False and this method fails, the method will
        return None.
    :return: data_matrix: M-by-N numpy array of actual data (yay!)
    :raises: ValueError: if this method fails and
        `raise_error_if_fails == True`.
    """

    # Check input args.
    num_grid_rows = int(numpy.round(num_grid_rows))
    num_grid_columns = int(numpy.round(num_grid_columns))
    field_name = str(field_name)
    wgrib2_exe_name = str(wgrib2_exe_name)
    raise_error_if_fails = bool(raise_error_if_fails)

    assert num_grid_rows > 0
    assert num_grid_columns > 0

    # Extract field from grib2 file to text file.
    temporary_file_name = tempfile.NamedTemporaryFile(
        dir=None, delete=False
    ).name

    command_string = (
        '"{0:s}" "{1:s}" -s | grep -w "{2:s}" | "{0:s}" -i "{1:s}" '
        '-no_header -text "{3:s}"'
    ).format(
        wgrib2_exe_name, grib2_file_name, field_name, temporary_file_name
    )

    print(command_string)

    try:
        subprocess.call(command_string, shell=True)
    except OSError as this_exception:
        os.remove(temporary_file_name)
        if raise_error_if_fails:
            raise

        warning_string = (
            '\n{0:s}\nCommand (shown above) failed (details shown below).'
            '\n{1:s}'
        ).format(
            command_string, str(this_exception)
        )

        warnings.warn(warning_string)
        return None

    # Load field into Python and reshape.
    data_values_1d = numpy.loadtxt(temporary_file_name)
    os.remove(temporary_file_name)

    try:
        data_matrix = numpy.reshape(
            data_values_1d, (num_grid_rows, num_grid_columns)
        )
    except ValueError as this_exception:
        if raise_error_if_fails:
            raise

        warning_string = (
            '\nnumpy.reshape failed (details shown below).\n{0:s}'
        ).format(
            str(this_exception)
        )

        warnings.warn(warning_string)
        return None

    return data_matrix

# Process data

The next cell processes data (namely, extracts predictor fields and saves them to an xarray table and then a NetCDF file) for the HRRR initialized at 2100 UTC 7 Sep 2022 and valid at 0000 UTC 8 Sep 2022 (a 3-hour forecast).

In [ ]:
MODEL_COORD_FILE_NAME = '/content/hrrr_data_for_notebook/HRRR_latlon.h5'
GRIB2_FILE_NAME = '/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2'
FORECAST_TIME_STRING = '2022-09-07-21'
VALID_TIME_STRING = '2022-09-08-00'

OUTPUT_FILE_NAME = (
    '/content/hrrr_data_for_notebook/hrrr_init={0:s}_valid={1:s}.nc'.format(
        FORECAST_TIME_STRING, VALID_TIME_STRING
    )
)

forecast_time_unix_sec = string_to_unix_sec(FORECAST_TIME_STRING, '%Y-%m-%d-%H')
valid_time_unix_sec = string_to_unix_sec(VALID_TIME_STRING, '%Y-%m-%d-%H')
attribute_dict = {
    FORECAST_TIME_KEY: forecast_time_unix_sec,
    VALID_TIME_KEY: valid_time_unix_sec
}

latitude_matrix_deg_n, longitude_matrix_deg_e = read_model_coords(
    MODEL_COORD_FILE_NAME
)
num_grid_rows = latitude_matrix_deg_n.shape[0]
num_grid_columns = latitude_matrix_deg_n.shape[1]

coord_dict = {
    ROW_DIMENSION: numpy.linspace(
        0, num_grid_rows - 1, num=num_grid_rows, dtype=int
    ),
    COLUMN_DIMENSION: numpy.linspace(
        0, num_grid_columns - 1, num=num_grid_columns, dtype=int
    ),
    PRESSURE_DIMENSION: PRESSURE_LEVELS_MB
}

main_data_dict = {
    LATITUDE_KEY: ((ROW_DIMENSION, COLUMN_DIMENSION), latitude_matrix_deg_n),
    LONGITUDE_KEY: ((ROW_DIMENSION, COLUMN_DIMENSION), longitude_matrix_deg_e)
}

num_pressure_levels = len(PRESSURE_LEVELS_MB)
num_3d_variables = len(VARIABLE_NAMES_3D)

for j in range(num_3d_variables):
    this_data_matrix = numpy.full(
        (num_grid_rows, num_grid_columns, num_pressure_levels), numpy.nan
    )
    
    for i in range(num_pressure_levels):
        this_field_name = '{0:s}:{1:d} mb'.format(
            VARIABLE_NAMES_3D_ORIG[j], PRESSURE_LEVELS_MB[i]
        )

        this_data_matrix[..., i] = read_field_from_grib2(
            grib2_file_name=GRIB2_FILE_NAME, field_name=this_field_name,
            num_grid_rows=num_grid_rows, num_grid_columns=num_grid_columns
        )

    print('Min and max values for {0:s} = {1:.2g}, {2:.2g}'.format(
        VARIABLE_NAMES_3D[j],
        numpy.min(this_data_matrix),
        numpy.max(this_data_matrix)
    ))

    main_data_dict[VARIABLE_NAMES_3D[j]] = (
        (ROW_DIMENSION, COLUMN_DIMENSION, PRESSURE_DIMENSION),
        this_data_matrix
    )

num_2d_variables = len(VARIABLE_NAMES_2D)

for j in range(num_2d_variables):
    this_data_matrix = read_field_from_grib2(
        grib2_file_name=GRIB2_FILE_NAME, field_name=VARIABLE_NAMES_2D_ORIG[j],
        num_grid_rows=num_grid_rows, num_grid_columns=num_grid_columns
    )

    this_data_matrix *= CONVERSION_RATES_2D[j]

    print('Min and max values for {0:s} = {1:.2g}, {2:.2g}'.format(
        VARIABLE_NAMES_2D[j],
        numpy.min(this_data_matrix),
        numpy.max(this_data_matrix)
    ))

    main_data_dict[VARIABLE_NAMES_2D[j]] = (
        (ROW_DIMENSION, COLUMN_DIMENSION),
        this_data_matrix
    )

hrrr_table_xarray = xarray.Dataset(
    data_vars=main_data_dict, coords=coord_dict, attrs=attribute_dict
)
print(hrrr_table_xarray)

hrrr_table_xarray.to_netcdf(
    path=OUTPUT_FILE_NAME, mode='w', format='NETCDF3_64BIT'
)

"/content/wgrib2_install/grib2/wgrib2/wgrib2" "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -s | grep -w "HGT:700 mb" | "/content/wgrib2_install/grib2/wgrib2/wgrib2" -i "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -no_header -text "/tmp/tmp09qfx3k4"
"/content/wgrib2_install/grib2/wgrib2/wgrib2" "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -s | grep -w "HGT:850 mb" | "/content/wgrib2_install/grib2/wgrib2/wgrib2" -i "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -no_header -text "/tmp/tmp6_w4jr38"
"/content/wgrib2_install/grib2/wgrib2/wgrib2" "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -s | grep -w "HGT:1000 mb" | "/content/wgrib2_install/grib2/wgrib2/wgrib2" -i "/content/hrrr_data_for_notebook/hrrr.t21z.wrfprsf03.grib2" -no_header -text "/tmp/tmp07chsspl"
Min and max values for geopotential_height_m_asl = -1.7e+02, 3.2e+03
"/content/wgrib2_install/grib2/wgrib2/wgrib2" "/content/hrrr_data_for_notebook/hrrr.t21z.wrfp